In [1]:
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Merging partitioned data loaded from ETL Pipeline

In [3]:
directory_path = "/content/drive/MyDrive/Merged_Data"

def paths(directory):
  files_path = []
  for root, dir, files in os.walk(directory_path):
    for file in files:

      if file.endswith(".csv"):
        files_path.append(os.path.join(root, file))

    return files_path

files_path = paths(directory_path)
print(len(files_path))

20


In [4]:
def read_and_join(list_path):
  df_container = []
  for file_path in list_path:

    df = pd.read_csv(file_path)
    df_container.append(df)
  merged_df = pd.concat(df_container, axis=0,ignore_index=True)
  return merged_df

df = read_and_join(files_path)
df.head(100)

,month,day_of_month,actual_departure_time,scheduled_departure_time,actual_arrival_time,scheduled_arrival_time,unique_carrier,flight_number,plane_tail_number,actual_elapsed_time,...,diverted,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,date,index,arrival_delay_indicator,departure_delay_indicator
0,6,22,1171.0,1160,1435.0,1430,WN,1446,NaN,144.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-06-22,510910,yes,yes
1,12,17,NaN,1070,NaN,1335,AA,614,0.0,NaN,...,0,0.0,0.0,0.0,0.0,0.0,2007-12-17,1074094,no,no
2,9,21,655.0,660,823.0,838,US,1953,NaN,108.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-09-21,799174,no,no
3,5,3,404.0,405,570.0,565,AA,365,NaN,226.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-05-03,374549,yes,no
4,8,8,966.0,966,1045.0,1050,FL,696,NaN,79.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-08-08,661681,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9,13,1155.0,1165,1270.0,1264,OH,5623,NaN,115.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-09-13,771675,yes,no
96,3,9,376.0,385,491.0,505,XE,2788,NaN,115.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-03-09,192596,no,no
97,3,8,548.0,540,721.0,716,US,570,NaN,113.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-03-08,198267,yes,yes
98,8,23,859.0,860,925.0,930,WN,696,NaN,66.0,...,0,0.0,0.0,0.0,0.0,0.0,2007-08-23,703501,no,no


In [6]:
df.to_csv("/content/drive/MyDrive/Analysis_Data/merged_data.csv")

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Analysis_Data/merged_data.csv")

In [7]:
df['date'] = pd.to_datetime(df['date'])
df_new = df[~df['date'].dt.year.isin([2005, 2006])]
df_new.shape

(9842432, 31)

In [8]:
cancelled_1 = df_new[df_new['cancelled']==1]
not_cancelled_1 = df_new[df_new['cancelled']==0]
print("cancelled",cancelled_1.shape)
print('not_cancelled',not_cancelled_1.shape)

cancelled (225190, 31)
not_cancelled (9617242, 31)


In [9]:
df_new.to_csv("/content/drive/MyDrive/Analysis_Data/new_data.csv")

## Dropping the following columns
- actual_departure_time
- scheduled_departure_time
- actual_arrival_time
- scheduled_arrival_time
- flight_number
- origin
- dest
- taxi_in
- taxi_out
- plane_tail_number
- cancellation_code
- diverted
- date
- index
- actual_elapse_time
- scheduled_elapse_time
- air_time
- arrival_delay
- departure_delay

In [11]:
df_1 = df_new.drop(columns = ['actual_elapsed_time','scheduled_elapsed_time','air_time','arrival_delay','departure_delay','actual_departure_time', 'scheduled_departure_time', 'actual_arrival_time', 'scheduled_arrival_time', 'flight_number', 'origin', 'dest', 'taxi_in', 'taxi_out', 'plane_tail_number', 'cancellation_code', 'diverted', 'date', 'index'], axis = 1)

In [12]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9842432 entries, 0 to 24124949
Data columns (total 12 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   month                      int64  
 1   day_of_month               int64  
 2   unique_carrier             object 
 3   distance                   int64  
 4   cancelled                  int64  
 5   carrier_delay              float64
 6   weather_delay              float64
 7   nas_delay                  float64
 8   security_delay             float64
 9   late_aircraft_delay        float64
 10  arrival_delay_indicator    object 
 11  departure_delay_indicator  object 
dtypes: float64(5), int64(4), object(3)
memory usage: 976.2+ MB


In [13]:
df_1.isnull().sum()

month                              0
day_of_month                       0
unique_carrier                     0
distance                           0
cancelled                          0
carrier_delay                1804634
weather_delay                1804634
nas_delay                    1804634
security_delay               1804634
late_aircraft_delay          1804634
arrival_delay_indicator            0
departure_delay_indicator          0
dtype: int64

### Dropping NaN Values

In [14]:
df_2 = df_1.dropna(subset=['carrier_delay','weather_delay','nas_delay','security_delay','late_aircraft_delay'])

In [15]:
df_2.shape

(8037798, 12)

In [16]:
cancelled_2 = df_2[df_2['cancelled']==1]
not_cancelled_2 = df_2[df_2['cancelled']==0]
print("cancelled rows",cancelled_2.shape)
print('not_cancelled rows',not_cancelled_2.shape)

cancelled rows (160748, 12)
not_cancelled rows (7877050, 12)


In [17]:
df_2.isnull().sum()

month                        0
day_of_month                 0
unique_carrier               0
distance                     0
cancelled                    0
carrier_delay                0
weather_delay                0
nas_delay                    0
security_delay               0
late_aircraft_delay          0
arrival_delay_indicator      0
departure_delay_indicator    0
dtype: int64

In [18]:
df_2

,month,day_of_month,unique_carrier,distance,cancelled,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,arrival_delay_indicator,departure_delay_indicator
0,6,22,WN,1037,0,0.0,0.0,0.0,0.0,0.0,yes,yes
1,12,17,AA,1562,1,0.0,0.0,0.0,0.0,0.0,no,no
2,9,21,US,599,0,0.0,0.0,0.0,0.0,0.0,no,no
3,5,3,AA,1372,0,0.0,0.0,0.0,0.0,0.0,yes,no
4,8,8,FL,406,0,0.0,0.0,0.0,0.0,0.0,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...
24124924,2,13,UA,733,0,3.0,0.0,0.0,0.0,31.0,yes,yes
24124931,1,5,YV,157,0,0.0,0.0,114.0,0.0,0.0,yes,yes
24124932,3,4,EV,644,0,0.0,0.0,104.0,0.0,0.0,yes,yes
24124935,4,10,WN,359,0,1.0,0.0,6.0,0.0,15.0,yes,yes


### Categorical Values

In [19]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder to label yes-->1 and no-->0
label_encoder = LabelEncoder()

df_2['arrival_delay_indicator'] = label_encoder.fit_transform(df_2['arrival_delay_indicator'])
df_2['departure_delay_indicator'] = label_encoder.fit_transform(df_2['departure_delay_indicator'])

<ipython-input-19-14bb4e8df2a8>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['arrival_delay_indicator'] = label_encoder.fit_transform(df_2['arrival_delay_indicator'])
<ipython-input-19-14bb4e8df2a8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2['departure_delay_indicator'] = label_encoder.fit_transform(df_2['departure_delay_indicator'])


In [21]:
# get_dummies function to perform one-hot encoding
df_encoded = pd.get_dummies(df_2, columns=['unique_carrier'])

In [23]:
df_encoded['arrival_delay_indicator']

0           1
1           0
2           0
3           1
4           0
           ..
24124924    1
24124931    1
24124932    1
24124935    1
24124938    1
Name: arrival_delay_indicator, Length: 8037798, dtype: int64

In [24]:
df_encoded.isnull().sum()

month                        0
day_of_month                 0
distance                     0
cancelled                    0
carrier_delay                0
weather_delay                0
nas_delay                    0
security_delay               0
late_aircraft_delay          0
arrival_delay_indicator      0
departure_delay_indicator    0
unique_carrier_9E            0
unique_carrier_AA            0
unique_carrier_AQ            0
unique_carrier_AS            0
unique_carrier_B6            0
unique_carrier_CO            0
unique_carrier_DL            0
unique_carrier_EV            0
unique_carrier_F9            0
unique_carrier_FL            0
unique_carrier_HA            0
unique_carrier_MQ            0
unique_carrier_NW            0
unique_carrier_OH            0
unique_carrier_OO            0
unique_carrier_UA            0
unique_carrier_US            0
unique_carrier_WN            0
unique_carrier_XE            0
unique_carrier_YV            0
dtype: int64

In [25]:
df_encoded['cancelled'].unique()

array([0, 1])

In [26]:
df_encoded.to_csv("/content/drive/MyDrive/Analysis_Data/encoded_data.csv")

In [27]:
data = pd.read_csv("/content/drive/MyDrive/Analysis_Data/encoded_data.csv")

In [28]:
data.shape

(8037798, 32)